## Notebook to test the bathymetry

In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import nc_time_axis

#plt.rcParams['axes.facecolor']='black'
#plt.style.use('dark_background')
plt.style.use('default')

plt.rcParams.update({'font.size': 10})

In [ ]:
path   = "/data/dkamm/nemo_output/NEVERWORLD/"

## Load domain (not lazy)

In [ ]:
from pathlib import Path
import xnemogcm as xn

In [ ]:
datadir_gaus    = Path(path + 'GAUS_RIDGE4')
datadir_cosh    = Path(path + 'COSH_RIDGE')
datadir_no      = Path(path + 'NO_RIDGE')

In [ ]:
domcfg_gaus = xn.open_domain_cfg(datadir=datadir_gaus)
domcfg_cosh = xn.open_domain_cfg(datadir=datadir_cosh)
domcfg_no   = xn.open_domain_cfg(datadir=datadir_no  )

### 1. Bathymetry without mid-atlantic ridge

In [ ]:
%matplotlib widget
fig = plt.figure()
ax  = plt.axes(projection='3d')
ax.set_box_aspect(aspect = (1,2.5,1))
(-domcfg_no.e3w_0.sum('z_f')).plot.surface(ax=ax)
ax.set_title('')

### 2. Bathymetry with mid-atlantic ridge:      Gaussian edge 

In [ ]:
%matplotlib widget
fig = plt.figure()
ax  = plt.axes(projection='3d')
ax.set_box_aspect(aspect = (1,2.5,1))
(-domcfg_gaus.e3w_0.sum('z_f')).plot.surface(ax=ax)
ax.set_title('')

### 3. Bathymetry without mid-atlantic ridge:   Cosh-shaped edge

In [ ]:
%matplotlib widget
fig = plt.figure()
ax  = plt.axes(projection='3d')
ax.set_box_aspect(aspect = (1,2.5,1))
(-domcfg_cosh.e3w_0.sum('z_f')).plot.surface(ax=ax)
ax.set_title('')

In [ ]:
import numpy as np

def gauss(rn_slp, rn_edge_n, phi, h_top, h_bot):
    if(h_bot >= h_top):
        gauss = (h_top - h_bot) * np.exp( - ( ( phi - rn_edge_n ) / rn_slp ) ** 2 ) + h_bot
    else:
        gauss = h_bot
    return gauss